In [1]:
import sys
import os

# Add Codes/src to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../src')))

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import random

from plot_utils import plot_comparison_boxplots
from train_utils import BatteryTrainerBase, BatterySeqDataset, build_lstm_sequences, pad_collate_fn


/Users/rizqipm/Documents/USYD_MDS/DATA5703_code/CS64-Capstone/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
df = pd.read_pickle('../../../Datasets/processed/merged_interp_flat.pkl')
df.head()

,cell_id,rated_capacity,cycle_index,capacity,soh,dqdt_min,dqdt_max,dqdt_mean,dqdt_std,dvdt_min,...,i_interp_991,i_interp_992,i_interp_993,i_interp_994,i_interp_995,i_interp_996,i_interp_997,i_interp_998,i_interp_999,source
0,B0005,2.0,1,1.856487,0.928244,-1.332504e-07,0.002876,0.000528,0.000255,-0.006229,...,0.004819,0.004902,0.004902,0.004902,0.004902,0.004902,0.004902,0.004902,0.004902,nasa
1,B0005,2.0,3,1.846327,0.923164,-4.510385e-07,0.002894,0.000527,0.000257,-0.006004,...,-0.000021,-0.000021,-0.000021,-0.000021,-0.000021,-0.000021,-0.000021,-0.000021,-0.000021,nasa
2,B0005,2.0,5,1.835349,0.917675,-1.163391e-07,0.002333,0.000520,0.000227,-0.005953,...,0.001754,0.001754,0.001754,0.001754,0.001754,0.001754,0.001754,0.001754,0.001754,nasa
3,B0005,2.0,7,1.835263,0.917631,-2.062661e-07,0.002885,0.000530,0.000255,-0.005907,...,0.002775,0.002775,0.002775,0.002775,0.002775,0.002775,0.002775,0.002775,0.002775,nasa
4,B0005,2.0,9,1.834646,0.917323,-5.353266e-07,0.002896,0.000530,0.000256,-0.005923,...,0.007981,0.007981,0.007981,0.007981,0.007981,0.007981,0.007981,0.007981,0.007981,nasa


In [ ]:
class BatteryTrainer(BatteryTrainerBase):
    def train(self, train_df, val_df, feature_cols=None, epochs=20, **model_kwargs):
        """
        Define your model here and how to train it
        should return the model y_true and y_pred
        e.g.:
            X_train = train_df[feature_cols].values
            y_train = train_df[self.target_cols].values
            X_val = val_df[feature_cols].values
            y_val = val_df[self.target_cols].values

            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)

            print(f"XGBOOST Val MSE={mean_squared_error(val_targets, val_preds):.4f}, MAE={mean_absolute_error(val_targets, val_preds):.4f}, R²={r2_score(val_targets, val_preds):.4f}")
            
            return model, y_val, y_pred
        """
        pass
    
    def optuna_objective(self, trial):
        """
        Define your hyperparameters here and return the objective value
        our recommendation is to return the avg_mse and std_mse 
        so that it will tune the model to minimize the mse and make the model more robust
        e.g.:
            n_estimators = trial.suggest_categorical("n_estimators", [100, 300])
            learning_rate = trial.suggest_categorical("learning_rate", [0.01, 0.1])
            max_depth = trial.suggest_categorical("max_depth", [3, 6])
            result = self.cross_validate(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                max_depth=max_depth,
            )
            
            # for robust model
            std_mse = np.std(result.mse)
            score = result.avg_mse + 0.8 * std_mse # we can tune the weight
            
            return score
        """
        pass

In [ ]:
trainer = BatteryTrainer(df, target_cols=['soh'])

In [ ]:
trainer.cross_validate()

In [ ]:
study = trainer.run_optuna_tuning()

In [ ]:
trainer.compare_base_vs_tuned_cv()

# Test Score

In [ ]:
base_model = trainer.train_base_model()

In [ ]:
tuned_model = trainer.train_tuned_model()